**Seminar:** Struktur und Interpretation von Computerprogrammen - Grundlegende Programmierkonzepte anhand der Sprache LISP, Frühjahrsemester 2021  
**Dozenten:** Prof. Dr. Christian Tschudin, Dr. Marcel Lüthi  
**Datum:** 5. September 2021  

# Eine Krivine-Maschine in Scheme  

Luka Obser <luka.obser@unibas.ch>  
Reto Krummenacher <reto.krummenacher@unibas.ch>

### Einleitung

#### Motivation
- curry-howard correspondence -> programs as proofs/proofs as programs, krivine asks "what are meaningful behaviours of
such programs and how can we evaluate them
- lambda calc as smallest formal system (3 rules for construction of terms and, in this case, 2 reduktion rules (alpha and beta reductions))

### 1. Theorie

#### 1.a) Lambda-Kalkül Recap
Rekursive Definition:

Variabel

Abstraktion

Applikation

#### 1.b)
alpha reduktion und de bruijn notation

ersetzen von variabeln durch "metainteger"

#### 1.c)
beta reduktion und call-by-name

### 2. Parsen und AST

### 3. Die Maschine
Die hier vorgestellte abstrakte Maschine in Scheme folgt der im original von Krivine postulierten Variante (Krivine 2007). In einem ersten Abschnitt werden die notwendigen Datenstrukturen vorgestellte, ehe auf das Verhalten der Maschiene mit ihrem Zustand und den Übergängen eingegangen wird.
#### 3.a) Datenstrukturen
Die Krivine-Machine benötigt drei Typen von Datenstrukturen: Closures, einen Stack und Environments. Im Folgenden wird vertieft auf diese drei Eingegangen. 
##### Eine Closure
Ein Closure ist ein Paar bestehend aus einem als Term bezeichneten ausführbaren $\lambda$-Ausdruck und einem Environment. Die Implementierung ist mit den notwendigen *getter* und *setter* Methoden versehen, wobei *'fst* und *'snd* das erste respektive das zweite Element eines Paars returnieren.

In [2]:
; closure data structure: a pair
(define (make-closure)
  (let ((c '()))
    (lambda (msg . args)
      (cond
        ((eq? msg 'set)
          (set! c (cons (car args) (cdr args))))
        ((eq? msg 'fst)
          (car c))
        ((eq? msg 'snd)
          (cadr c))
        ((eq? msg 'get) c)
        (else
          (display (string-append "\nERROR: Not supported command: " (symbol->string msg))) (newline))))))  

##### Der Stack
Auf dem Stack befinden sich Closures. Implementiert wurde dieser in Scheme als Liste. Mittels den üblichen Befehlen *'push* und *'pop* können Closures auf den Stack geschoben oder vom Stack geholt werden. Ebenfalls enthalten ist die Methode *'display*. Damit ist es möglich die einzelnen Closures auf dem Stack in der Konsole auszugeben.

In [3]:
; stack data structure: a list with push and pop
(define (make-stack)
  (let ((s '()) (n 0))
     (lambda (msg . args)  ; msg and arguments, if . used, args is seen as list which can be empty
       (cond
         ((eq? msg 'pop)
            (cond
              ((null? s)
                (display "\nERROR: Stack is empty\n"))
              (else
                (set! n (- n 1))
                (define tmpS s)
                (set! s (cdr s))
                (car tmpS)))) ;return the car element of stack
         ((eq? msg 'push)
            (set! n (+ n 1))
            (set! s (append (reverse args) s)))
         ((eq? msg 'get) s)
         ((eq? msg 'size) n)
         ((eq? msg 'display)
            (define clos (make-closure))
            (let loop ((i 0))
               (cond ((< i (length s))
                   (set! clos (list-ref s i))
                   (display "[") (display (clos 'fst)) (display ",") (display (clos 'snd)) (display "] ")
                   (loop (+ i 1))))))
         (else
          (display (string-append "\nERROR: Not supported command: " (symbol->string msg))) (newline))))))

##### Ein Environment
Ein Environment wurde ebenfalls als Liste implementiert. Wenn ein Environment nicht leer ist, dann ist das erste Element selbst ein Environment. Alle folgenden sind Closures, welche mittels *'append* angefügt werden können. Die Datenstruktur bietet die Möglichkeit entweder das enthaltene Environment oder die Closure an Position *k* zrückzugeben.

In [ ]:
; environment data structure: a list
(define (make-environment)
  (let ((e '()))
    (lambda (msg . args)
      (cond
        ((eq? msg 'append)
          (set! e (append e args)))  ; append takes list, thus no (car args) needed
        ((eq? msg 'getHigh)
          (car e))
        ((eq? msg 'getK)
          (list-ref e (car args))) ; get the kth closure of this environment
        ((eq? msg 'get) e)        
        (else
          (display (string-append "\nERROR: Not supported command: " (symbol->string msg))) (newline))))))

#### 3.b) Zustand
Der Zustand der Krivine-Maschine ist das Tripel besthenden aus Term $T$, Stack $S$ und Environment $E$. Der Term ist der als nächstes auszuführende $\lambda$-Ausdruck. $E$ und $S$ sind zu Beginn leer.  
Im Unterschied zur ursprünglichen Idee von Krivine (2007, S. 201), wird in der hier präsentierten Implementierung nicht mit Zeigern gearbeitet. Dies ist der Tatsache geschuldet, dass die Sprache Scheme im Gegensatz etwa zur Sprache C keine Zeigervariablen kennt. Daher wird direkt mit den Objekten gearbeitet.  
Um einen Zustand festzuhalten, wurde die Datenstruktur *krivine-state* geschreiben, welche für alle drei Elemente des Zustands eine ensprechende *'get* und *'set* Methode anbietet.

In [7]:
; Krivine Machine state consisting of next to evaluate term T, stack S and environment E
; on object creation elements are initialized
(define (krivine-state)
  (let ((T '()) (S (make-stack)) (E (make-environment)))
    (lambda (msg . args)  ; args is always a list, to get sinlge element use (car args)
       (cond
          ((eq? msg 'setT) (set! T (car args)))
          ((eq? msg 'setS) (set! S (car args))) 
          ((eq? msg 'setE) (set! E (car args))) 
          ((eq? msg 'getT) T)
          ((eq? msg 'getS) S)
          ((eq? msg 'getE) E)
          (else
             (display (string-append "\nERROR: Not supported command: " (symbol->string msg))) (newline))))))

#### 3.c) Zustandsveränderung
Krivine (2007, S. 201/204) beschreibt drei Zustandsveränderung in Abhängigkeit vom aussehen des Terms $T$, also des auszuführenden Ausdrucks. Im vorliegenden Fall wurden die Regeln derart implementiert, dass sie eine State-Objekt als Argument entgegen nehmen, die notwendigen Schritt ausführen und danach den aktualisierten Zustand zurückgeben. Es sind die die Applikation, die Abstraktion und das Ausführen einer Variable.
##### Applikation
Eine Applikation liegt vor, wenn $T=(u)x$ ist. Der Ausdruck $x$ wird zusammen mit dem aktuellen Environment $E$ in einer Closure $\xi=<x,E>$ gespeichert und diese anschliessend auf den Stack gepushed. Für den nächsten Schritt wird $T=u$ gesetzt. Das Environment bleibt unverändert.

In [4]:
; 1. Application of the form (λx.x)(λy.y): APP (ABS (#\λ . 1) VAR . "x") ABS (#\λ . 1) VAR . "z")
;    Create closure with the address of the latter part (e.g. λy.y), push to stack and update T
;    @Param: a state object
;    @Return: a state object
(define (app s) 
  (define clos (make-closure))
  (clos 'set (cddr (s 'getT)) (s 'getE)) ; create closure with second part of T pair as new T and the current environment
  (define stack (s 'getS))
  (stack 'push clos)
  (s 'setS stack)  ; push it to stack
  (s 'setT (cadr (s 'getT))) ; T is the first element of the pair which is evaluated next
  s) ; resturn new state

##### Abstraktion
Eine Abstaktion liegt vor, wenn der Term mit einem $\lambda$ beginnt, also zum Beispiel $T=\lambda x_{1}\ldots\lambda x_{n}y$ ist und $y$ kein $\lambda$ vorangestellt hat. In diesem Fall wird ein neues Environment $E'$ erstellt. Das erste Element ist das bisherige $E$. Die weiteren Elemente sind die $n$ Closures, welche vom Stack geholt werden. Somit ist $E'=(E,\xi_1,\ldots,\xi_n)$. Für die Weiterverarbeitung ist $T=y$ und $E=E'$. Der Stack kann, muss aber nicht leer sein.

In [5]:
; 2. Abstraction λx.x: (ABS (#\λ . 1) VAR . "u")
;    Create new environment e.g (E, clos1, ..., closN), by popping n times from stack
;    @Param: a state object
;    @Return: a state object
(define (abs s)
  (define env (make-environment))
  (env 'append (s 'getE)) ; update E, the pointer to current environment
  (define stack (s 'getS))
  (let loop ((i (cdr (cadr (s 'getT))))) ; do number of pops
     (cond ((> i 0)
         (env 'append (stack 'pop))
         (loop (- i 1)))))
  (s 'setE env) ; this is new state environment
  (s 'setS stack) ; new stack
  (s 'setT (cddr (s 'getT))) ;T becomes the last element of former T pair
   s) ; return new state

##### Ausführen einer Variable
Wenn der Term nur eine einzelne Variable enthält, denn wird wiefolgt vorgegangen.  
Die Variable wurde im Kompilierschritt mit einem Wertepaar $<v,k>$ ersetzt. Dies gibt an in welchem Environment in welcher Closure die auszuführende Variable gebunden ist. Zuerst wird das richtige Enviroment gesucht. Ist $v=0$, dann handelt es sich um das aktuelle $E$. Falls $v>0$ wird das nächst höhere Environment $E_1$ gesucht, sprich das erste Element von $E$. Dies wird wiederholt, bis $E_v$ gefunden ist.  
Der Wert $k$ gibt an, welches Closure $\xi_k$ aus $E_v$ benötigt wird. Eine Closure besteht aus einem Term und einem Environment. $T$ wird nun zum Term von $\xi_k$ und $E$ zum Environment von $\xi_k$.

In [6]:
; 3. Variable x
;    The Value of x was replaced by an ordered pair <v,k>
;    The v gives the environment in which x can be found
;    k is the number of the closure inside that environment
;    @Param: a state object
;    @Return: a state object
(define (var s)
  (define env (s 'getE))
  (define e (make-environment))
  (let ((v (cadr (s 'getT))) (k (cddr (s 'getT))))
    ;(display (string-append "v:" (number->string v) " k:" (number->string k))) (newline)
    ; get recursive the environment v and update state
    (let loop ((i 0))
     (cond ((< i v)          
         (set! e (env 'getHigh))
         (set! env e)
         (cond ((null? env)
             (error "Empty environment found. Machine stopped" (env 'get))))         
         (loop (+ i 1)))))
    ; get k closure in the updated state environment and update T, check if this is available
    (cond ((> k (length(env 'get)))
               (error "No Closure with needed index in environment. Machine stopped" (env 'get)))
          (else
               (let ((T '()) (E '()) (clos (make-closure)))
                   (set! clos (env 'getK k))
                   (set! T (clos 'fst))
                   (set! E (clos 'snd))
                   (s 'setT T) ; elements of clos are T and E
                   (s 'setE E))
               s)))) ; return new state

#### 3.d) Die Krivine-Routine
Mittels der vorgestellten Regeln kann nun die Krivine-Maschine mittels rekursivem Aufruf simuliert werden. Die Maschine ist fertig, wenn $S$ und $E$ wieder leer sind. Dann ist das Ergebnis in $T$ enthalten. 

In [8]:
; Krivine Machine
(define (krivine-machine T)
  ; create state object
  (define state (krivine-state))
  (state 'setT T)
  (define n 0)
  ; eval loop
  (let eval ((state state) (n n))
     (display (string-append "State after step " (number->string n) ":")) (newline) 
     (display "T: ") (display (state 'getT)) (newline)
     (display "S: ") ((state 'getS) 'display) (newline)
     (display "E: ") (display ((state 'getE) 'get)) (newline) (newline)
     (cond ((eq? (car (state 'getT)) 'APP)
            (eval (app state) (+ n 1)))
           ((and (not (null? ((state 'getS) 'get))) (eq? (car (state 'getT)) 'ABS))
            ; start abstraction only if stack is non empty
            (eval (abs state) (+ n 1)))
           ((eq? (car (state 'getT)) 'VAR)
            (eval (var state) (+ n 1)))
           ((and (null? ((state 'getS) 'get)) (null? ((state 'getE) 'get)))
            state)  ; termination condition
           ))
  (state 'getT))

Hier ist Test:

### Fazit

## Quellenverzeichnis
- Krivine, Jean-Louis (2007):   
   *A call-by-name lambda-calculus machine*, Springer, <https://link.springer.com/article/10.1007/s10990-007-9018-9> (letzter Zugriff 3.9.2021)
- Rojas, Raúl (2015):  
  *A Tutorial Introduction to the Lambda Calculus*, FU Berlin, <https://arxiv.org/abs/1503.09060> (letzter Zugriff 3.9.2021)
- De Bruijn, Nicolaas Govert (1972):  
  *Lambda calculus notation with nameless dummies, a tool for automatic formula manipulation, with application to the Church-Rosser theorem*, <https://doi.org/10.1016/1385-7258(72)90034-0> (letzter Zugriff 3.9.2021)